### Parte 2. 
Problema, Encontrar para el estado de Sonora, la relacion entre promedio de escolaridad y Salarios en el estado, por sexo

En esta libreta, realizaremos revisión, limpieza de datos,        
Los datos de inegi traen  bastante información por lo que se crearán tablas que mantengan las columnas relevantes y se trabajará con estas.

In [45]:
import pandas as pd
import numpy as np
import os
import json
import openpyxl

from pandas_profiling import ProfileReport

ModuleNotFoundError: No module named 'openpyxl'

In [2]:
#extracting data from Json file
# load data using Python JSON module
with open("./datamex/datamexicowage.json",'r') as f:
    data = json.loads(f.read())
# Flatten data
df_waged = pd.json_normalize(data, record_path =['data'])
df_waged.head()

,Sex ID,Sex,State ID,State,State ES,Age Range ID,Age Range,Wage
0,1,Male,26,Sonora,Sonora,4,15 to 19 years,935.077755
1,1,Male,26,Sonora,Sonora,5,20 to 24 years,2414.132528
2,1,Male,26,Sonora,Sonora,6,25 to 29 years,2922.294776
3,1,Male,26,Sonora,Sonora,7,30 to 34 years,3987.337161
4,1,Male,26,Sonora,Sonora,8,35 to 39 years,5036.093005


In [3]:
df_waged.info() #check extracted df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sex ID        30 non-null     int64  
 1   Sex           30 non-null     object 
 2   State ID      30 non-null     int64  
 3   State         30 non-null     object 
 4   State ES      30 non-null     object 
 5   Age Range ID  30 non-null     int64  
 6   Age Range     30 non-null     object 
 7   Wage          28 non-null     float64
dtypes: float64(1), int64(3), object(4)
memory usage: 2.0+ KB


create wages_tidy

In [4]:
df_waged['State'].equals(df_waged['State ES']) #verify if State and Stte ES contain the same values, result true

#our new dataframe will drop some columns , and rows, since we will focus on Sonora data
wages_columns=  ['Sex ID',
                 'Sex',
                 'Age Range',
                 'State',
                 'Wage']

wages_tidy = df_waged[wages_columns].copy() #creating a copy for tidy data
wages_tidy.shape

(30, 5)

In [5]:
wages_tidy.isnull().sum() # check for null values

Sex ID       0
Sex          0
Age Range    0
State        0
Wage         2
dtype: int64

In [6]:
#Identify NA Values to
wages_tidy[wages_tidy['Wage'].isna()] 

,Sex ID,Sex,Age Range,State,Wage
14,1,Male,85 or more years,Sonora,NaN
28,2,Female,80 to 84 years,Sonora,NaN


In [7]:
wages_tidy=wages_tidy.dropna(subset=['Wage']) # Remove NaN values in this case, due to low count of NA
wages_tidy[wages_tidy['Wage'].isna()] 


,Sex ID,Sex,Age Range,State,Wage


In [8]:
wages_tidy.head()

,Sex ID,Sex,Age Range,State,Wage
0,1,Male,15 to 19 years,Sonora,935.077755
1,1,Male,20 to 24 years,Sonora,2414.132528
2,1,Male,25 to 29 years,Sonora,2922.294776
3,1,Male,30 to 34 years,Sonora,3987.337161
4,1,Male,35 to 39 years,Sonora,5036.093005


In [9]:
wages_tidy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 29
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Sex ID     28 non-null     int64  
 1   Sex        28 non-null     object 
 2   Age Range  28 non-null     object 
 3   State      28 non-null     object 
 4   Wage       28 non-null     float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.3+ KB


Para las bases de Inegi, se mantendrán solo las columnas referentes a rangos de edad y educación, esto se busca en los diccionarios de datos originales, y esta información se agregará  a las bases modificadas.


In [10]:
#Extracting data from 2020 and 2010 files

df2020 = pd.read_csv("./inegi_2020/iter_26_cpv2020/conjunto_de_datos/conjunto_de_datos_iter_26CSV20.csv")
df2020.shape
df2010 = pd.read_csv("./inegi_2020/iter_26_cpv2020/conjunto_de_datos/conjunto_de_datos_iter_26CSV20.csv")

In [11]:
df2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Columns: 286 entries, ENTIDAD to TAMLOC
dtypes: float64(1), int64(6), object(279)
memory usage: 16.4+ MB


In [12]:
#our new dataframe will drop some columns , and rows, since we will focus on data related to education
son2020 =  [
            'NOM_MUN',
            'POBTOT',
            'POBFEM',
            'POBMAS',
            'P_15A19_F',
            'P_15A19_M',
            'P_20A24_F',
            'P_20A24_M',
            'P_25A29_F',
            'P_25A29_M',
            'P_30A34_F',
            'P_30A34_M',
            'P_35A39_F',
            'P_35A39_M',
            'P_40A44_F',
            'P_40A44_M',
            'P_45A49_F',
            'P_45A49_M',
            'P_50A54_F',
            'P_50A54_M',
            'P_55A59_F',
            'P_55A59_M',
            'P_60A64_F',
            'P_60A64_M',
            'P_65A69_F',
            'P_65A69_M',
            'REL_H_M',
            'GRAPROES',
            'GRAPROES_F',
            'GRAPROES_M']

son2020_edu = df2020[son2020].copy() #creating a copy for education data
son2020_edu.describe

<bound method NDFrame.describe of                          NOM_MUN   POBTOT   POBFEM   POBMAS P_15A19_F  \
0     Total de la entidad Sonora  2944840  1472643  1472197    124961   
1     Total de la entidad Sonora    16883     5317    11566       393   
2     Total de la entidad Sonora     5087     2155     2932       168   
3                        Aconchi     2563     1227     1336       105   
4                        Aconchi     1650      785      865        63   
...                          ...      ...      ...      ...       ...   
7495      San Ignacio Río Muerto        2        *        *         *   
7496      San Ignacio Río Muerto        2        *        *         *   
7497      San Ignacio Río Muerto        1        *        *         *   
7498      San Ignacio Río Muerto      296      128      168         8   
7499      San Ignacio Río Muerto       60       27       33         1   

     P_15A19_M P_20A24_F P_20A24_M P_25A29_F P_25A29_M  ... P_55A59_F  \
0       130755  

In [13]:
son2020_edu['NOM_MUN'] = son2020_edu['NOM_MUN'].astype('string')
son2020_edu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   NOM_MUN     7500 non-null   string
 1   POBTOT      7500 non-null   int64 
 2   POBFEM      7500 non-null   object
 3   POBMAS      7500 non-null   object
 4   P_15A19_F   7500 non-null   object
 5   P_15A19_M   7500 non-null   object
 6   P_20A24_F   7500 non-null   object
 7   P_20A24_M   7500 non-null   object
 8   P_25A29_F   7500 non-null   object
 9   P_25A29_M   7500 non-null   object
 10  P_30A34_F   7500 non-null   object
 11  P_30A34_M   7500 non-null   object
 12  P_35A39_F   7500 non-null   object
 13  P_35A39_M   7500 non-null   object
 14  P_40A44_F   7500 non-null   object
 15  P_40A44_M   7500 non-null   object
 16  P_45A49_F   7500 non-null   object
 17  P_45A49_M   7500 non-null   object
 18  P_50A54_F   7500 non-null   object
 19  P_50A54_M   7500 non-null   object
 20  P_55A59_

In [14]:
#convert objects to float
obj_columns = ['POBFEM',
               'POBMAS',
               'P_15A19_F',
               'P_15A19_M',
               'P_20A24_F',
               'P_20A24_M',
               'P_25A29_F',
               'P_25A29_M',
               'P_30A34_F',
               'P_30A34_M',
               'P_35A39_F',
               'P_35A39_M',
               'P_40A44_F',
               'P_40A44_M',
               'P_45A49_F',
               'P_45A49_M',
               'P_50A54_F',
               'P_50A54_M',
               'P_55A59_F',
               'P_55A59_M',
               'P_60A64_F',
               'P_60A64_M',
               'P_65A69_F',
               'P_65A69_M',
               'REL_H_M',
               'GRAPROES',
               'GRAPROES_F',
               'GRAPROES_M']

son2020_edu[obj_columns] = son2020_edu[obj_columns].apply(pd.to_numeric, errors='coerce') # convert object columns to float
son2020_edu.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 30 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   NOM_MUN     7500 non-null   string 
 1   POBTOT      7500 non-null   int64  
 2   POBFEM      2068 non-null   float64
 3   POBMAS      2068 non-null   float64
 4   P_15A19_F   2037 non-null   float64
 5   P_15A19_M   2037 non-null   float64
 6   P_20A24_F   2037 non-null   float64
 7   P_20A24_M   2037 non-null   float64
 8   P_25A29_F   2037 non-null   float64
 9   P_25A29_M   2037 non-null   float64
 10  P_30A34_F   2037 non-null   float64
 11  P_30A34_M   2037 non-null   float64
 12  P_35A39_F   2037 non-null   float64
 13  P_35A39_M   2037 non-null   float64
 14  P_40A44_F   2037 non-null   float64
 15  P_40A44_M   2037 non-null   float64
 16  P_45A49_F   2037 non-null   float64
 17  P_45A49_M   2037 non-null   float64
 18  P_50A54_F   2037 non-null   float64
 19  P_50A54_M   2037 non-null  

-Al realizar  un examen preliminar, del dataframe con las columnas seleccionadas, se encuentra que algunos valores del municipio se repiten (inicialmente para este análisis se descarto localidad) por lo que es necesario agregar los municipios, lo que se realiza en los siguientes pasos 


In [15]:

son2020_2 = son2020_edu.groupby('NOM_MUN').sum() #Group by "Nombre del Municipio"
son2020_2

,POBTOT,POBFEM,POBMAS,P_15A19_F,P_15A19_M,P_20A24_F,P_20A24_M,P_25A29_F,P_25A29_M,P_30A34_F,...,P_55A59_F,P_55A59_M,P_60A64_F,P_60A64_M,P_65A69_F,P_65A69_M,REL_H_M,GRAPROES,GRAPROES_F,GRAPROES_M
NOM_MUN,,,,,,,,,,,,,,,,,,,,,
Aconchi,5137,2454.0,2672.0,210.0,214.0,144.0,200.0,152.0,178.0,164.0,...,136.0,122.0,122.0,118.0,120.0,130.0,713.20,50.37,53.47,47.62
Agua Prieta,184138,92236.0,91622.0,8628.0,8932.0,7982.0,7966.0,7112.0,6962.0,6954.0,...,3708.0,3786.0,3294.0,2972.0,2026.0,2132.0,1842.73,113.28,116.26,111.15
Alamos,50413,24134.0,25818.0,2012.0,2252.0,1686.0,1812.0,1540.0,1494.0,1510.0,...,1326.0,1284.0,1198.0,1336.0,858.0,1006.0,18275.10,945.17,954.85,929.28
Altar,19217,9560.0,9424.0,948.0,996.0,808.0,724.0,686.0,640.0,622.0,...,374.0,354.0,288.0,292.0,236.0,196.0,1040.11,91.87,84.77,91.07
Arivechi,2356,1165.0,1187.0,80.0,78.0,62.0,30.0,60.0,70.0,70.0,...,68.0,75.0,68.0,78.0,64.0,48.0,663.92,40.69,39.96,41.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tubutama,3036,1380.0,1566.0,106.0,118.0,88.0,98.0,88.0,78.0,88.0,...,86.0,86.0,84.0,108.0,62.0,74.0,2451.75,133.65,129.57,130.05
Ures,17399,8194.0,8902.0,604.0,670.0,420.0,442.0,462.0,472.0,472.0,...,536.0,652.0,528.0,604.0,440.0,422.0,2444.13,148.21,146.57,148.61
Villa Hidalgo,2859,1361.0,1496.0,100.0,94.0,60.0,62.0,64.0,48.0,70.0,...,68.0,68.0,74.0,110.0,94.0,94.0,326.59,23.44,23.50,23.42


In [16]:
son2020_2.isnull().sum() # check for null values

POBTOT        0
POBFEM        0
POBMAS        0
P_15A19_F     0
P_15A19_M     0
P_20A24_F     0
P_20A24_M     0
P_25A29_F     0
P_25A29_M     0
P_30A34_F     0
P_30A34_M     0
P_35A39_F     0
P_35A39_M     0
P_40A44_F     0
P_40A44_M     0
P_45A49_F     0
P_45A49_M     0
P_50A54_F     0
P_50A54_M     0
P_55A59_F     0
P_55A59_M     0
P_60A64_F     0
P_60A64_M     0
P_65A69_F     0
P_65A69_M     0
REL_H_M       0
GRAPROES      0
GRAPROES_F    0
GRAPROES_M    0
dtype: int64

In [17]:
son2020_2.head()

,POBTOT,POBFEM,POBMAS,P_15A19_F,P_15A19_M,P_20A24_F,P_20A24_M,P_25A29_F,P_25A29_M,P_30A34_F,...,P_55A59_F,P_55A59_M,P_60A64_F,P_60A64_M,P_65A69_F,P_65A69_M,REL_H_M,GRAPROES,GRAPROES_F,GRAPROES_M
NOM_MUN,,,,,,,,,,,,,,,,,,,,,
Aconchi,5137,2454.0,2672.0,210.0,214.0,144.0,200.0,152.0,178.0,164.0,...,136.0,122.0,122.0,118.0,120.0,130.0,713.20,50.37,53.47,47.62
Agua Prieta,184138,92236.0,91622.0,8628.0,8932.0,7982.0,7966.0,7112.0,6962.0,6954.0,...,3708.0,3786.0,3294.0,2972.0,2026.0,2132.0,1842.73,113.28,116.26,111.15
Alamos,50413,24134.0,25818.0,2012.0,2252.0,1686.0,1812.0,1540.0,1494.0,1510.0,...,1326.0,1284.0,1198.0,1336.0,858.0,1006.0,18275.10,945.17,954.85,929.28
Altar,19217,9560.0,9424.0,948.0,996.0,808.0,724.0,686.0,640.0,622.0,...,374.0,354.0,288.0,292.0,236.0,196.0,1040.11,91.87,84.77,91.07
Arivechi,2356,1165.0,1187.0,80.0,78.0,62.0,30.0,60.0,70.0,70.0,...,68.0,75.0,68.0,78.0,64.0,48.0,663.92,40.69,39.96,41.44


In [18]:
son2020_2.shape

(73, 29)

Como siguiente paso, alinearemos la info de las tablas para unirlas con los rangos de edad del dataframe de Inegi, para que coincidan con lo de nuestra tabla `wages_tidy`

In [19]:
#to continue to align dataframes, we are going to transpose the columns of Sonora DataFrame
son2020_t = son2020_2.T['Total de la entidad Sonora'] # On this section we will use Only the totals
son2020_t = son2020_t.reset_index()
son2020_t.head()


,index,Total de la entidad Sonora
0,POBTOT,2966810.0
1,POBFEM,1480115.0
2,POBMAS,1486695.0
3,P_15A19_F,125522.0
4,P_15A19_M,131699.0


In [20]:
son2020_t.columns

Index(['index', 'Total de la entidad Sonora'], dtype='object')

In [21]:
#convert to DF And rename column
#son2020_t = son2020_t.to_frame()
son2020_t.rename(columns={son2020_t.columns[0]: 'Age_range'},inplace=True)


In [22]:
print(wages_tidy["Age Range"].unique()) #Identify unique ranges available in Wages


['15 to 19 years' '20 to 24 years' '25 to 29 years' '30 to 34 years'
 '35 to 39 years' '40 to 44 years' '45 to 49 years' '50 to 54 years'
 '55 to 59 years' '60 to 64 years' '65 to 69 years' '70 to 74 years'
 '75 to 79 years' '80 to 84 years' '85 or more years']


In [23]:
#Modify rows to match Sonora Dataframe
wages_tidy['Age Range'] = wages_tidy['Age Range'].map(lambda x: x.lstrip('+-').rstrip('years'))
wages_tidy['Age Range']

0       15 to 19 
1       20 to 24 
2       25 to 29 
3       30 to 34 
4       35 to 39 
5       40 to 44 
6       45 to 49 
7       50 to 54 
8       55 to 59 
9       60 to 64 
10      65 to 69 
11      70 to 74 
12      75 to 79 
13      80 to 84 
15      15 to 19 
16      20 to 24 
17      25 to 29 
18      30 to 34 
19      35 to 39 
20      40 to 44 
21      45 to 49 
22      50 to 54 
23      55 to 59 
24      60 to 64 
25      65 to 69 
26      70 to 74 
27      75 to 79 
29    85 or more 
Name: Age Range, dtype: object

In [24]:
#Modify rows to match Sonora Dataframe
wages_tidy['Age Range'] = wages_tidy['Age Range'].str.replace(r' to ', 'A')
wages_tidy.rename(columns={wages_tidy.columns[2]: 'Age_range'},inplace=True)
wages_tidy['Age_range'] =  "P_" + wages_tidy.Age_range
wages_tidy.loc[wages_tidy['Sex'].str.startswith('Male'), 'Age_range'] = wages_tidy['Age_range'] + '_M'
wages_tidy.loc[wages_tidy['Sex'].str.startswith('Female'), 'Age_range'] = wages_tidy['Age_range'] + '_F'


In [32]:
ob_columns = ['Sex',
              'State'
            ]
wages_tidy[ob_columns] = wages_tidy[ob_columns].astype('|S') # convert object columns to string
wages_tidy.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 29
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Sex ID     28 non-null     int64  
 1   Sex        28 non-null     |S6    
 2   Age_range  28 non-null     object 
 3   State      28 non-null     |S6    
 4   Wage       28 non-null     float64
dtypes: bytes48(2), float64(1), int64(1), object(1)
memory usage: 2.2+ KB


Unir `Dataframes`

In [25]:

datos_concat = pd.concat([son2020_t, wages_tidy ], axis = 1, join="inner")
datos_concat


,Age_range,Total de la entidad Sonora,Sex ID,Sex,Age_range,State,Wage
0,POBTOT,2966810.00,1,Male,P_15A19 _M,Sonora,935.077755
1,POBFEM,1480115.00,1,Male,P_20A24 _M,Sonora,2414.132528
2,POBMAS,1486695.00,1,Male,P_25A29 _M,Sonora,2922.294776
3,P_15A19_F,125522.00,1,Male,P_30A34 _M,Sonora,3987.337161
4,P_15A19_M,131699.00,1,Male,P_35A39 _M,Sonora,5036.093005
5,P_20A24_F,122059.00,1,Male,P_40A44 _M,Sonora,5683.256573
6,P_20A24_M,126973.00,1,Male,P_45A49 _M,Sonora,2863.591427
7,P_25A29_F,114250.00,1,Male,P_50A54 _M,Sonora,3110.581985
8,P_25A29_M,115592.00,1,Male,P_55A59 _M,Sonora,4926.737791
9,P_30A34_F,108005.00,1,Male,P_60A64 _M,Sonora,2266.266506


### Crear Diccionarios de Datos.
Se crearan diccionarios para las tablas principales

In [37]:
wages_tidy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 29
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Sex ID     28 non-null     int64  
 1   Sex        28 non-null     |S6    
 2   Age_range  28 non-null     object 
 3   State      28 non-null     |S6    
 4   Wage       28 non-null     float64
dtypes: bytes48(2), float64(1), int64(1), object(1)
memory usage: 2.2+ KB


In [38]:
class create_data_dictionary:

    def __init__(self):
        '''This class provides functions to quickly develop a data dictionary for your data set'''
        return None

    def make_my_data_dictionary(self, dataFrame):
        '''Create an initial data dictionary excluding definitions for meaning of features'''

        col_ = dataFrame.columns
        df_DataDict = {}

        for col in col_:
                df_DataDict[col] = {
                               'Type': str(df.dtypes[col]),
                               'Length': len(df[col]),
                               'Null_Count': sum(df[col].isna()),
                               'Size(Memory)': df.memory_usage()[col],
                               'Description': str('')
                                }

        df_DD = pd.DataFrame(df_DataDict)

        return df_DD

    def define_data_meaning(self, df_data_dictionary):
        '''Quickly provide input regarding each columns meaning and transpose into a usable dictionary'''

        col_ = df_data_dictionary.columns
        d = 'Definition'

        for col in col_:
            df_data_dictionary[col][d] = input('Provide a data definition for {}'.format(col))

        df_data_dictionary = df_data_dictionary.transpose()

        return df_data_dictionary

    def update_dd_definition(self, df_data_dictionary, attribute):
        try:
            df_dd = df_data_dictionary.transpose()
            df_dd[attribute]['Definition'] = input('Provide a data definition for {}'.format(attribute))
            df_dd = df_dd.transpose()
            return df_dd
        except:
            print('Sorry, there was an error.  Check attribute name and try again')


In [39]:
df = wages_tidy

In [40]:
dd = create_data_dictionary()
df_dd = dd.make_my_data_dictionary(df)
df_dd = dd.define_data_meaning(df_dd)


In [44]:
#Create excel writer object and send DF to excel
createfile = pd.ExcelWriter('diccionario01.xlsx')
df_dd.to_excel(createfile)

createfile.save()
print("Dataframe  saved to file")

ModuleNotFoundError: No module named 'openpyxl'